In [22]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [23]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [24]:
pip install pymongo

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [25]:
pip install lxml

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [26]:
from bs4 import BeautifulSoup
import pandas as pd
import pymongo

In [63]:
import requests
import json
import time
import copy

In [28]:
#proof of concept testing
def get_artist_data_from_itunes(url):
    response = requests.get(url).json()
    result_set = response["results"]
    list_out = []
    for album_info in result_set:
        album_dict = {}
        album_dict['AlbumName'] = album_info['collectionName']
        album_dict['Artist'] = album_info['artistName']
        album_dict['Explicit'] = album_info['collectionExplicitness']
        album_dict['Genre'] = album_info['primaryGenreName']
        album_dict['Price'] = album_info['collectionPrice']
        album_dict['TrackCount'] = album_info['trackCount']
        list_out.append(album_dict)
    return list_out

In [29]:
url = "https://itunes.apple.com/search?term=jack+johnson&entity=album"

In [30]:
artist_data=get_artist_data_from_itunes(url)

In [31]:
artist_df = pd.DataFrame(artist_data)

In [32]:
print(artist_df)

                                            AlbumName  \
0             In Between Dreams (Bonus Track Version)   
1   Brushfire Fairytales (Remastered) [Bonus Version]   
2                            Sleep Through the Static   
3                    To the Sea (Bonus Track Version)   
4                                           On and On   
5                             From Here To Now To You   
6   Jack Johnson & Friends - Best of Kokua Festiva...   
7                          All the Light Above It Too   
8                                   En Concert (Live)   
9   Jack Johnson And Friends: Sing-A-Longs And Lul...   
10                                          On and On   
11  Jack Johnson and Friends: Sing-A-Longs and Lul...   
12  Sing-A-Longs & Lullabies for the Film Curious ...   
13  Rudolph the Red Nosed Reindeer (Live from Late...   
14                                  In Between Dreams   
15               Willie Got Me Stoned (Live) - Single   
16         Farewell (From "El M

In [33]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [34]:
db = client.billboard_db
collection = db.items

In [35]:
# URL of page to be scraped
url = 'https://www.billboard.com/charts/artist-100'

# Retrieve page with the requests module
response = requests.get(url)
# Create BeautifulSoup object; parse with 'lxml'
soup = BeautifulSoup(response.text, 'lxml')
print(soup.prettify)

<bound method Tag.prettify of <!DOCTYPE html>
<html class="" lang="">
<head>
<script>
        _udn = "billboard.com";
    </script>
<script>function utmx_section(){}function utmx(){}(function(){var
                k='67942495-8',d=document,l=d.location,c=d.cookie;
            if(l.search.indexOf('utm_expid='+k)>0)return;
            function f(n){if(c){var i=c.indexOf(n+'=');if(i>-1){var j=c.
                    indexOf(';',i);return escape(c.substring(i+n.length+1,j<0?c.
                    length:j))}}}var x=f('__utmx'),xx=f('__utmxx'),h=l.hash;d.write(
                    '<sc'+'ript src="'+'http'+(l.protocol=='https:'?'s://ssl':
                            '://www')+'.google-analytics.com/ga_exp.js?'+'utmxkey='+k+
                            '&utmx='+(x?x:'')+'&utmxx='+(xx?xx:'')+'&utmxtime='+new Date().
                            valueOf()+(h?'&utmxhash='+escape(h.substr(1)):'')+
                            '" type="text/javascript" charset="utf-8"><\/sc'+'ript>')})();
    </scri

In [36]:
# Examine the results, then determine element that contains sought info
# results are returned as an iterable list
artist_list = soup.find_all(class_='chart-list-item')
artist_names=[]
for artist in artist_list:
    try:
        artist_names.append(artist.a.text.strip())
    except:
        artist_names.append(artist.span.text.strip())

print(artist_names)   



['Tool', 'Taylor Swift', 'Post Malone', 'Lana Del Rey', 'Lizzo', 'Billie Eilish', 'Khalid', 'BTS', 'Lil Tecca', 'Ed Sheeran', 'Shawn Mendes', 'Drake', 'Luke Combs', 'Jonas Brothers', 'Young Thug', 'Ariana Grande', 'Lil Nas X', 'Camila Cabello', 'DaBaby', 'Travis Scott', 'Queen', 'Imagine Dragons', 'Lewis Capaldi', 'Chris Brown', 'Dan + Shay', 'Cardi B', 'Panic! At The Disco', 'Kane Brown', 'Lil Baby', 'Sam Smith', 'Halsey', '5 Seconds Of Summer', 'Justin Bieber', 'Lauren Daigle', 'Normani', 'Bruno Mars', 'Luke Bryan', 'Juice WRLD', 'Blanco Brown', 'Sheryl Crow', 'Bad Bunny', 'NF', 'Megan Thee Stallion', 'Marshmello', 'P!nk', 'Jason Aldean', 'Blake Shelton', 'J. Cole', 'Elton John', 'Thomas Rhett', 'Katy Perry', 'Saweetie', 'Morgan Wallen', 'J Balvin', 'Maroon 5', 'XXXTENTACION', 'Polo G', 'Bon Iver', 'Trisha Yearwood', 'Swae Lee', 'Carrie Underwood', 'SHAED', 'twenty one pilots', 'Stick Figure', 'Miley Cyrus', 'Maren Morris', 'Nicki Minaj', 'Chris Stapleton', 'The Chainsmokers', 'Emine

In [37]:
#deals with char. differences between artist names, e.g. Motley Crue (imagine the umlauts in there), u2, Charlie Daniels Band
#'jack+johnson' is the standard format of artist firstname+lastname
def transform_artist_names(in_list):
    out_list = []
    for name in in_list:
        split_string=name.replace(' ','+')
        out= split_string.lower()
        out_list.append(out)
    return out_list

In [38]:
new_names=transform_artist_names(artist_names)

In [39]:
print(new_names)

['tool', 'taylor+swift', 'post+malone', 'lana+del+rey', 'lizzo', 'billie+eilish', 'khalid', 'bts', 'lil+tecca', 'ed+sheeran', 'shawn+mendes', 'drake', 'luke+combs', 'jonas+brothers', 'young+thug', 'ariana+grande', 'lil+nas+x', 'camila+cabello', 'dababy', 'travis+scott', 'queen', 'imagine+dragons', 'lewis+capaldi', 'chris+brown', 'dan+++shay', 'cardi+b', 'panic!+at+the+disco', 'kane+brown', 'lil+baby', 'sam+smith', 'halsey', '5+seconds+of+summer', 'justin+bieber', 'lauren+daigle', 'normani', 'bruno+mars', 'luke+bryan', 'juice+wrld', 'blanco+brown', 'sheryl+crow', 'bad+bunny', 'nf', 'megan+thee+stallion', 'marshmello', 'p!nk', 'jason+aldean', 'blake+shelton', 'j.+cole', 'elton+john', 'thomas+rhett', 'katy+perry', 'saweetie', 'morgan+wallen', 'j+balvin', 'maroon+5', 'xxxtentacion', 'polo+g', 'bon+iver', 'trisha+yearwood', 'swae+lee', 'carrie+underwood', 'shaed', 'twenty+one+pilots', 'stick+figure', 'miley+cyrus', 'maren+morris', 'nicki+minaj', 'chris+stapleton', 'the+chainsmokers', 'emine

In [65]:
# Starting URL for iTunes API Call
url = "https://itunes.apple.com/search?term=" 

# List of name data
new_names_data = []

# Print to logger
print("Pulling from iTunes API")
print("-----------------------------")

# Create counters
record_count = 1
set_count = 1
fulllist_out = []

# Loop through all the names in our list
for i, name in enumerate(new_names):
        
   
    # Create endpoint URL with each city
#     name_url = url + new_names + "&entity=album"
    name_url = url + name + "&entity=album"
    
    # Log the url, record, and set numbers
#     print("Processing Record %s of Set %s | %s" % (record_count, set_count, name))

    # Add 1 to the record count
    record_count += 1
    
#     def get_artist_data_from_itunes(name_url):
    response = requests.get(name_url).json()
    fullresult_set = response["results"]
#         return fulllist_out
    for album_info in fullresult_set:
            album_dict = {}
            album_dict['AlbumName'] = album_info['collectionName']
            album_dict['Artist'] = album_info['artistName']
            album_dict['Explicit'] = album_info['collectionExplicitness']
            album_dict['Genre'] = album_info['primaryGenreName']
#             album_dict['Price'] = album_info['collectionPrice']
            album_dict['TrackCount'] = album_info['trackCount']
            fulllist_out.append(album_dict)
#     print(fulllist_out)
    
    if record_count % 15 == 0:
        time.sleep(30)
        
#     return fulllist_out
print(len(fulllist_out))
# Indicate that Data Loading is complete 
print("-----------------------------")
print("iTunes API pull complete")
print("-----------------------------")
        
        

Pulling from iTunes API
-----------------------------
3518
-----------------------------
iTunes API pull complete
-----------------------------


In [66]:
# Convert array of JSONs into Pandas DataFrame
new_names_data = pd.DataFrame(fulllist_out)

# Extract relevant fields from the data frame
#lats = city_data_pd["Lat"]
#max_temps = city_data_pd["Max Temp"]
#humidity = city_data_pd["Humidity"]
#cloudiness = city_data_pd["Cloudiness"]
#wind_speed = city_data_pd["Wind Speed"]

# Export the City_Data into a csv
#city_data_pd.to_csv(output_data_file, index_label="City_ID")

# Show Record Count
#city_data_pd.count()
new_names_data

,AlbumName,Artist,Explicit,Genre,TrackCount
0,Fear Inoculum,TOOL,explicit,Metal,10
1,Ænima,TOOL,explicit,Metal,15
2,Undertow,TOOL,explicit,Metal,10
3,Lateralus,TOOL,notExplicit,Metal,13
4,"10,000 Days",TOOL,explicit,Metal,11
5,Opiate - EP,TOOL,explicit,Metal,6
6,IllutionWorld - EP,TOOL$,notExplicit,Hip-Hop/Rap,4
7,Every Breath You Take (Club Mix) - Single,"Shannon, Tool & Vektor",notExplicit,House,1
8,The String Quartet Tribute to Tool's Aenima,Vitamin String Quartet,notExplicit,Rock,15
9,The Ultimate Tool,8-Bit Arcade,notExplicit,Electronica,34


In [68]:
new_names_data.to_csv("Artist.csv", index=False, header=True)